In [19]:
import ast
import os
import pandas as pd

# get all full texts
list_of_files = []
for index in range(1, 13):
    if os.path.isfile(f"../data/pmc_patients/processed/full_texts_{index}.csv"):
        df = pd.read_csv(f"../data/pmc_patients/processed/full_texts_1.csv")
        print(len(df))
        list_of_files.append(df)

2945
2945
2945
2945
2945
2945
2945
2945
2945
2945
2945
2945


In [25]:
# combine all full texts and safe to csv
df = pd.concat(list_of_files)
print(len(df))

# delete rows with empty full text
df = df.dropna(subset=["full_text"])
print(len(df))

# remove rows with failed downloads
df = df[~df.full_text.str.contains("Failed to download")]

# update index
df = df.reset_index(drop=True)

print(len(df))
print(df.head(10))

35340
35028
29124
      PMID                                          full_text
0   203128  THE YALE JOURNAL OF BIOLOGY AND MEDICINE 50 (1...
1   743489  Br. J. Cancer (1978) 38, 719\nCLINICAL RESPONS...
2   749356  THE YALE JOURNAL OF BIOLOGY AND MEDICINE 51 (1...
3   962990  Br. J. (Cancer (1976) 34, 111\nASSOCIATION OF ...
4  1520599  Br.\nJ.\nCancer\n(1992),\n66,\n603\n606\n1992\...
5  1556154  Characterization ofa Novel Peripheral Nervous\...
6  1701825  Brief Definitive Report\nThe Effect of Immunos...
7  1733434  Br. J. Cancer (1992), 65, 114-117\n©\nMacmilla...
8  2391360  Fimbrin Is a Homologue of the Cytoplasmic Phos...
9  2432071  Acidic and Basic Hair/Nail ("Hard") Keratins: ...


In [26]:
df.to_csv("../data/pmc_patients/processed/full_texts_combined.csv", index=False)

## Remove patients with no full text available

In [74]:
# get all full text paper ids
full_text_paper_ids = df["PMID"].tolist()

# get all relevant articles
list_of_articles = pd.read_csv("../data/pmc_patients/Summary_data/list_of_articles.csv")
print(len(list_of_articles))

# drop articles that are not in the list of full text papers
list_of_articles = list_of_articles[list_of_articles["article"].isin(full_text_paper_ids)]

print(len(list_of_articles))
list_of_articles.reset_index(drop=False, inplace=True)
list_of_articles.to_csv("../data/pmc_patients/Summary_data/list_of_articles_with_full_text.csv", index=False)

128836
1828


In [94]:
import ast

patients = pd.read_csv("../data/pmc_patients/Summary_data/patient_relevant_articles_map.csv")
list_of_papers = list_of_articles["article"].tolist()
def flatten_list(input_list):
    return [item for sublist in input_list for item in sublist]

# drop articles that are not in the list of full text papers
def filter_relevant_articles(relevant_articles):
    articles = ast.literal_eval(relevant_articles)
    return [article for article in articles if int(article) in list_of_papers]

patients["relevant_articles"] = patients["relevant_articles"].apply(filter_relevant_articles)
# drop row if relevant articles are empty
patients = patients[patients["relevant_articles"].apply(len) > 0]
patients.reset_index(drop=False, inplace=True)
patients.to_csv("../data/pmc_patients/Summary_data/patient_relevant_articles_map_with_full_text.csv", index=False)

In [95]:
patients

,index,patient,relevant_articles
0,0,This 60-year-old male was hospitalized due to ...,[12493078]
1,1,A 39-year-old man was hospitalized due to an i...,[12493078]
2,2,One week after a positive COVID-19 result this...,[12493078]
3,3,This 69-year-old male was admitted to the ICU ...,[12493078]
4,4,This 57-year-old male was admitted to the ICU ...,[12493078]
...,...,...,...
1974,9977,A 55-year-old man with a history of type II di...,[19543490]
1975,9981,A 15-year-old female presented with heat intol...,[21928702]
1976,9982,A 21-year-old female presented with fatigue an...,[21928702]
1977,9983,A 39-year-old female presented with fatigue an...,[21928702]
